# Capstone Project

This is notebook contains the Capstone project for the [IBM Data Science course](https://www.coursera.org/professional-certificates/ibm-data-science).

## Segmenting and Clustering Neighborhoods in Toronto

Import necessary libraries

In [1]:
import pandas as pd
import html5lib

Read and load in the data from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

table = pd.read_html(url)[0]

Read data into Pandas Dataframe

In [3]:
df = pd.DataFrame(table)
df = df.rename(columns=df.iloc[0]).drop(df.index[0])

Drop rows where Borough = 'Not assigned'

In [4]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)

Group rows by Postcode and reindex

In [5]:
df = df.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ', '.join(x))
df.reset_index(level=['Postcode','Borough'], inplace=True)

Copy Borough into Neighbourhood where Neighbourhood = 'Not assigned'

In [6]:
df['Neighbourhood'].loc[df['Neighbourhood'] == "Not assigned"] = df['Borough']

Display Dataframe

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Read in lat & long coordinates from CSV

In [8]:
coord = pd.read_csv('Geospatial_Coordinates.csv')

Check that coordinates have been loaded correctly

In [9]:
coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge location dataframe with lat & long coordinates on the Postcode column

In [10]:
df = df.merge(coord, how='left').reset_index(drop=True)

Display Dataframe

In [11]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Check Dataframe shape

In [12]:
df.shape

(103, 5)